In [1]:
from glob import glob
import pandas as pd
import xarray as xr
from pismragis.processing import convert_netcdf_to_dataframe, ncfile2dataframe
from pismragis.analysis import prepare_df
from pismragis.analysis import prepare_df, sensitivity_analysis


In [ ]:
infiles = glob("../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_2020-1-1.nc")
df = convert_netcdf_to_dataframe(infiles, add_vars=False)
df.to_parquet("../tests/data/test_scalar.parquet")
df.to_csv("../tests/data/test_scalar.csv")



In [ ]:
df = convert_netcdf_to_dataframe(infiles, add_vars=False, resample="yearly")
df.to_parquet("../tests/data/test_scalar_YM.parquet")
df.to_csv("../tests/data/test_scalar_YM.csv")



In [ ]:
df = convert_netcdf_to_dataframe(infiles, add_vars=True)
df.to_parquet("../tests/data/test_scalar_add_vars.parquet")



In [ ]:
infile = "../tests/data/ts_gris_g1200m_v2023_RAGIS_id_0_1980-1-1_2020-1-1.nc"

In [ ]:
df = ncfile2dataframe(infile, add_vars=False)
df.to_parquet("../tests/data/test_scalar_file.parquet")
df.to_csv("../tests/data/test_scalar_file.csv")

In [ ]:
df = ncfile2dataframe(infile, add_vars=False, resample="yearly")
df.to_parquet("../tests/data/test_scalar_file_YM.parquet")
df.to_csv("../tests/data/test_scalar_file_YM.csv")

In [ ]:
df = ncfile2dataframe(infile, add_vars=True)
df.to_parquet("../tests/data/test_scalar_file_add_vars.parquet")
df.to_csv("../tests/data/test_scalar_file_add_vars.csv")

In [3]:
X_df = pd.read_parquet("tests/data/test_scalar_YM.parquet")

In [2]:
cd ../

/Users/andy/base/pism-ragis


In [ ]:
X_df

In [7]:
df = X_df[X_df.time.between("1980-01-01", "1983-01-01")]
df.to_csv("tests/data/test_scalar_YM.csv")
df.to_parquet("tests/data/test_scalar_YM.parquet")

In [4]:
    sens_vars = [
        "vcm",
        "gamma_T",
        "thickness_calving_threshold",
        "ocean_file",
        "sia_e",
        "ssa_n",
        "pseudo_plastic_q",
        "till_effective_fraction_overburden",
        "phi_min",
        "phi_max",
        "z_min",
        "z_max",
    ]


In [5]:
    ensemble_file = "tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"


In [9]:
n_jobs=4
Y_true = sensitivity_analysis(df, ensemble_file=ensemble_file, n_jobs=n_jobs)[
            sens_vars
        ]

-------------------------------------------



Processing date: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.77s/it]

Program finished in 7 seconds


In [ ]:
Y_true.to_parquet("tests/data/test_sensitivity.parquet")

In [ ]:
ds = xr.open_mfdataset("tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc", combine="nested", concat_dim="id", parallel=True)

In [ ]:
ens_vars = ["grounding_line_flux", "limnsw"]
ens = ds.sel(time=slice("1980-01-01" ,"1983-01-01"))[ens_vars].resample(time="1AS").mean()
ens_df = ens.to_dataframe().reset_index().dropna()

In [ ]:
ens_vars

In [ ]:
n_jobs=4

Y_true_xr = sensitivity_analysis(ens_df, ensemble_file=ensemble_file, n_jobs=n_jobs, calc_variables=ens_vars)[sens_vars]

In [ ]:
ens_df

In [ ]:
Y_true.mean()

In [ ]:
Y_true_xr.mean()

In [ ]:
Y_true

In [10]:
import numpy as np

for i in [0, 1, 2]:
    sa_df = sensitivity_analysis(df, ensemble_file=ensemble_file, n_jobs=n_jobs)
    print(sa_df.head())
#sa_xr_df = sensitivity_analysis(ens_df, ensemble_file=ensemble_file, n_jobs=n_jobs, calc_variables=ens_vars)

-------------------------------------------



Processing date: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.27s/it]


Program finished in 5 seconds
        vcm   gamma_T  thickness_calving_threshold  ocean_file     sia_e  \
0  0.013839  0.076529                     0.150908    0.022933  0.012045   
1  0.116679  0.117876                     0.142859    0.109067  0.106948   
2  0.035965  0.083400                     0.487787    0.060835  0.022225   
3  0.215805  0.311129                     0.344166    0.231794  0.197514   
4  0.007037  0.107412                     0.124952    0.023995 -0.011007   

      ssa_n  pseudo_plastic_q  till_effective_fraction_overburden   phi_min  \
0  0.071812          0.163356                            0.028859  0.012032   
1  0.111776          0.161075                            0.094437  0.097658   
2  0.077961          0.394478                            0.016439  0.067883   
3  0.249993          0.343488                            0.184276  0.250863   
4  0.081200          0.183455                            0.014589  0.020615   

    phi_max     z_min     z_max       

Processing date: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.32s/it]


Program finished in 5 seconds
        vcm   gamma_T  thickness_calving_threshold  ocean_file     sia_e  \
0  0.014829  0.082239                     0.152717    0.010225  0.011004   
1  0.083192  0.091442                     0.125830    0.106542  0.118775   
2  0.035965  0.083400                     0.487787    0.060835  0.022225   
3  0.200042  0.184842                     0.298431    0.243837  0.193664   
4  0.020590  0.105547                     0.125738    0.025267 -0.006003   

      ssa_n  pseudo_plastic_q  till_effective_fraction_overburden   phi_min  \
0  0.068437          0.167467                            0.024545  0.000450   
1  0.116316          0.147741                            0.106973  0.130008   
2  0.077961          0.394478                            0.016439  0.067883   
3  0.296014          0.351895                            0.206558  0.272467   
4  0.079235          0.191064                            0.008733  0.012526   

    phi_max     z_min     z_max       

Processing date: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.31s/it]

Program finished in 5 seconds
        vcm   gamma_T  thickness_calving_threshold  ocean_file     sia_e  \
0  0.020121  0.075449                     0.141537    0.016628  0.014923   
1  0.093605  0.107794                     0.130017    0.107947  0.103229   
2  0.035965  0.083400                     0.487787    0.060835  0.022225   
3  0.208215  0.216037                     0.320676    0.255384  0.193645   
4  0.011953  0.104878                     0.134963    0.011353 -0.003305   

      ssa_n  pseudo_plastic_q  till_effective_fraction_overburden   phi_min  \
0  0.067340          0.166507                            0.036994  0.005611   
1  0.114644          0.150337                            0.093691  0.104514   
2  0.077961          0.394478                            0.016439  0.067883   
3  0.284382          0.356252                            0.180394  0.276374   
4  0.082605          0.172728                            0.008825  0.014381   

    phi_max     z_min     z_max       

In [ ]:
sa_df.head()

In [ ]:
sa_xr_df.head()

In [13]:
import pathlib
import time
from typing import Union

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from pandas.api.types import is_string_dtype
from SALib.analyze import delta
from scipy.interpolate import interp1d
from tqdm import tqdm

from pismragis.processing import tqdm_joblib

np.random.seed(42)


In [14]:
    df = X_df[X_df.time.between("1980-01-01", "1983-01-01")]
    id_df = (pd.read_csv(ensemble_file) * 1).replace(np.nan, 0)

    param_names = id_df.drop(columns="id").columns.values.tolist()
    for k, col in id_df.items():
        if is_string_dtype(col):
            u = col.unique()
            u.sort()
            v = [k for k, v in enumerate(u)]
            col.replace(to_replace=dict(zip(u, v)), inplace=True)
    # Define a salib "problem"
    problem = {
        "num_vars": len(id_df.drop(columns="id").columns.values),
        "names": param_names,  # Parameter names
        "bounds": zip(
            id_df.drop(columns="id").min().values,
            id_df.drop(columns="id").max().values,
        ),  # Parameter bounds
    }

    df = pd.merge(id_df, df, on="id")
    # filter out dates with only 1 experiment, e.g., due to
    # CDO changing the mid-point time when running averaging
    df = pd.concat([x for _, x in df.groupby(by="time") if len(x) > 1])
    n_dates = len(df["time"].unique())


In [15]:
for m_date, s_df in df.groupby(by="time"):

,id,vcm,gamma_T,thickness_calving_threshold,ocean_file,sia_e,ssa_n,pseudo_plastic_q,till_effective_fraction_overburden,phi_min,phi_max,z_min,z_max,time,grounding_line_flux (Gt year-1),limnsw (kg),Year,resolution_m
0,0,0.748460,0.000141,208.554189,6,2.458823,3.272571,0.641733,0.017322,6.841640,42.734234,-504.83478,387.58102,1980-01-01,-495.444694,2.633911e+18,1980.0,1200
4,1,0.797112,0.000136,254.360985,4,2.645698,3.338459,0.532839,0.021179,5.372954,41.244648,-563.01120,297.82916,1980-01-01,-621.209970,2.633845e+18,1980.0,1200
8,2,0.837068,0.000147,369.085887,6,1.687356,3.307447,0.839391,0.014796,7.444413,40.183514,-422.70566,188.01651,1980-01-01,-506.279400,2.633912e+18,1980.0,1200
12,3,0.496831,0.000107,182.910783,3,2.807056,3.331697,0.729520,0.017594,8.992171,42.957695,-291.17220,182.28912,1980-01-01,-405.928142,2.633959e+18,1980.0,1200
16,4,0.638088,0.000149,256.354402,7,2.435586,3.300404,0.825152,0.016005,8.666372,41.940525,-310.17828,187.55905,1980-01-01,-410.013025,2.633960e+18,1980.0,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,20,0.922305,0.000150,278.450143,10,2.915390,3.314241,0.783855,0.018274,6.441045,42.750538,-376.46097,176.82220,1983-01-01,-421.261802,2.633968e+18,1983.0,1200
83,21,0.721245,0.000122,237.770934,8,2.356902,3.353182,0.855321,0.016030,7.917657,44.619293,-310.34393,168.30064,1983-01-01,-352.617843,2.634201e+18,1983.0,1200
87,22,0.466431,0.000133,215.754677,0,2.475785,3.291879,0.612520,0.018178,12.318474,42.700626,-192.64525,176.90793,1983-01-01,-546.570516,2.633486e+18,1983.0,1200
91,23,0.597726,0.000139,281.354326,8,2.930306,3.258649,0.612399,0.023160,7.282847,41.633663,-414.54898,360.99924,1983-01-01,-509.886912,2.633556e+18,1983.0,1200


In [16]:
    missing_ids = list(set(id_df["id"]).difference(s_df["id"]))
    if missing_ids:
        if verbose:
            print(f"The following simulation ids are missing:\n   {missing_ids}")

        id_df_missing_removed = id_df[~id_df["id"].isin(missing_ids)]
        params = np.array(
            id_df_missing_removed.drop(columns="id").values, dtype=np.float32
        )
    else:
        params = np.array(id_df.drop(columns="id").values, dtype=np.float32)
        Si = delta.analyze(
            problem,
            params,
            response_matrix,
            num_resamples=100,
            print_to_console=False,
        )
        Si_df = Si.to_df()


NameError: name 's_df' is not defined